##### Manually insert generation parameters

In [1]:
from cfdmod.use_cases.block_gen import *
from cfdmod.use_cases.block_gen import SpacingParams

block_params = BlockParams(
    height=5,
    width=5,
    length=5
)

spacing_params = SpacingParams(
    spacing_x=2, 
    spacing_y=2, 
    line_offset=5, 
    is_abs=False, 
    offset_direction="x"
)

cfg = GenerationParams(
    N_blocks_x=10,
    N_blocks_y=10,
    block_params=block_params,
    spacing_params=spacing_params
)

cfg

GenerationParams(N_blocks_x=10, N_blocks_y=10, block_params=BlockParams(height=5.0, width=5.0, length=5.0), spacing_params=SpacingParams(spacing_x=2.0, spacing_y=2.0, line_offset=5.0, is_abs=False, offset_direction=<OffsetDirection.x: 'x'>))

##### Calculate spacings and offset from parameters

In [3]:
single_line_repeat = (
    cfg.N_blocks_y - 1
    if cfg.spacing_params.offset_direction == "y"
    else cfg.N_blocks_x - 1
)
single_line_spacing = (
    cfg.spacing_params.spacing_x + cfg.block_params.length
    if cfg.spacing_params.offset_direction == "x"
    else cfg.spacing_params.spacing_y + cfg.block_params.width
)
multiple_line_repeat = (
    cfg.N_blocks_x - 1
    if cfg.spacing_params.offset_direction == "y"
    else cfg.N_blocks_y - 1
)
multiple_line_spacing = (
    cfg.spacing_params.spacing_x + cfg.block_params.length
    if cfg.spacing_params.offset_direction == "y"
    else cfg.spacing_params.spacing_y + cfg.block_params.width
)

single_line_repeat, single_line_spacing, multiple_line_repeat, multiple_line_spacing

(9, 7.0, 9, 7.0)

##### Build block and linear patterns

In [8]:
vertices, triangles = build_single_block(cfg.block_params)

single_line_vertices, single_line_triangles = linear_pattern(
    vertices,
    triangles,
    direction=cfg.spacing_params.offset_direction,
    n_repeats=single_line_repeat,
    spacing_value=single_line_spacing,
)

full_vertices, full_triangles = linear_pattern(
    single_line_vertices,
    single_line_triangles,
    direction=cfg.perpendicular_direction,
    n_repeats=multiple_line_repeat,
    spacing_value=multiple_line_spacing,
    offset_value=cfg.calculate_spacing(direction=cfg.perpendicular_direction),
)

print("Single block:", f"Vertices count: {len(vertices)}", f"Triangles count: {len(triangles)}")
print("Single line", f"Vertices count: {len(single_line_vertices)}", f"Triangles count: {len(single_line_triangles)}")
print("Replicated lines", f"Vertices count: {len(full_vertices)}", f"Triangles count: {len(full_triangles)}")

Single block: Vertices count: 8 Triangles count: 10
Single line Vertices count: 80 Triangles count: 100
Replicated lines Vertices count: 800 Triangles count: 1000


##### Export generated geometry

In [9]:
from cfdmod.api.geometry.STL import export_stl
import pathlib

output_path = pathlib.Path("./output/block_gen")

export_stl(output_path / "blocks.stl", full_vertices, full_triangles)